Python code: [mandelbrot_gpu.py](https://github.com/edwardchalstrey1/turingbench/blob/master/turingbench_python_cuda/mandelbrot_gpu/mandelbrot_gpu.py) - creates mandelbrot image, using GPU, see how the non-GPU version looks [in this notebook](https://github.com/edwardchalstrey1/turingbench/blob/master/turingbench_python_cuda/simple_mandelbrot/mandelbrot%20time%20container.ipynb).

The below Dockerfile is modified from: https://github.com/Luke035/nvidia-anaconda-docker/blob/master/Dockerfile

```docker build -t edwardchalstrey/mandelbrot_gpu .```

In [1]:
%%writefile Dockerfile
FROM nvidia/cuda:9.0-cudnn7-runtime-ubuntu16.04

RUN  apt-get update \
  && apt-get install -y wget vim bzip2\
  && rm -rf /var/lib/apt/lists/*

RUN apt-get update
RUN apt-get -y install curl

#Install MINICONDA
RUN wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O Miniconda.sh && \
    /bin/bash Miniconda.sh -b -p /opt/conda && \
    rm Miniconda.sh

ENV PATH /opt/conda/bin:$PATH

RUN conda install numpy scipy matplotlib numba cudatoolkit=9.0 pyculib -y

COPY mandelbrot_gpu.py /mandelbrot_gpu.py

CMD python3 mandelbrot_gpu.py

Overwriting Dockerfile


In [2]:
%%writefile mandelbrot_gpu.def
BootStrap: docker 
From: edwardchalstrey/mandelbrot_gpu

%post
    apt-get -y update

%files      
    mandelbrot_gpu.py /mandelbrot_gpu.py

Overwriting mandelbrot_gpu.def


In [3]:
%%writefile jade_sub.sh
#!/bin/bash

# set the number of nodes
#SBATCH --nodes=1

# set max wallclock time
#SBATCH --time=10:00:00

# set name of job
#SBATCH --job-name=echalstrey_singularity_cuda_test1

# set number of GPUs
#SBATCH --gres=gpu:4

# mail alert at start, end and abortion of execution
#SBATCH --mail-type=ALL

# send mail to this address
#SBATCH --mail-user=echalstrey@turing.ac.uk

# run the application
module load singularity
singularity build mandelbrot_gpu.sif mandelbrot_gpu.def
singularity run --nv mandelbrot_gpu.sif

Writing jade_sub.sh


The Docker image has been built and pushed to [Docker Hub](https://cloud.docker.com/u/edwardchalstrey/repository/docker/edwardchalstrey/mandelbrot_gpu):

1. ```docker build -t edwardchalstrey/mandelbrot_gpu .```
2. ```docker push edwardchalstrey/mandelbrot_gpu```

Then can be run with Docker: ```docker run edwardchalstrey/mandelbrot_gpu```

Singularity Commands to build from the Docker Hub image and run:

1. ```singularity build mandelbrot_gpu.sif mandelbrot_gpu.def```
2. ```singularity run --nv mandelbrot_gpu.sif```

*Note, building singularity images appear to take up a lot of storage and might be difficult on Azure VMs with the default 30GB*

| Platform  | Container  | Mandelbrot creation time in s  |
|---|---|---|
| Azure (Nvidia K80)  | Docker  | 5.369710  |
| Azure (Nvidia K80)  | Singularity 3.2  | 5.734466  |
| CSD3 (Nvidia V100)  | Singularity  |   |
| JADE (Nvidia P100)  | Singularity 2.4 |   |